# Nomotic Quickstart

**Nomotic** is a runtime governance framework for agentic AI. Every action an AI agent attempts is evaluated across **13 governance dimensions** before, during, and after execution. Governance is not advisory -- it is authoritative.

In this notebook you will learn to:

1. **Create a governance runtime** -- the single entry point for all governance decisions
2. **Evaluate actions** -- pass actions through the full 13-dimension pipeline and receive verdicts
3. **Observe trust dynamics** -- watch trust build incrementally as an agent demonstrates good behavior
4. **Test boundaries** -- see what happens when an agent attempts an out-of-scope action
5. **Inspect audit trails** -- review the structured, append-only log of every governance event

In [ ]:
# Install nomotic from the local repository
!pip install -e "."

In [ ]:
# Birth an agent via the Nomotic CLI
!nomotic birth --name notebook-agent --role assistant

In [ ]:
from nomotic import GovernanceRuntime, Action, AgentContext, TrustProfile, Verdict

# Create a governance runtime -- the orchestrator for all governance decisions
runtime = GovernanceRuntime()

# Configure the scope for our agent: allow read, write, and query actions
runtime.configure_scope("notebook-agent", {"read", "write", "query"})

# Create an action to evaluate
action = Action(
    agent_id="notebook-agent",
    action_type="read",
    target="documents",
)

# Create an agent context with a trust profile
context = AgentContext(
    agent_id="notebook-agent",
    trust_profile=TrustProfile(agent_id="notebook-agent"),
)

# Evaluate the action through the full governance pipeline
verdict = runtime.evaluate(action, context)

print(f"Verdict:         {verdict.verdict.name}")
print(f"UCS Score:       {verdict.ucs:.4f}")
print(f"Tier:            {verdict.tier}")
print(f"Evaluation Time: {verdict.evaluation_time_ms:.2f} ms")

In [ ]:
# Trust dynamics: run 10 successful read actions and watch trust build
#
# Trust is calibrated continuously. Each successful action increases trust
# by a small increment (default: +0.01). Building trust is slow by design --
# losing it is fast.

print(f"{'Action':>6}  {'Trust Score':>12}  {'Verdict':>8}")
print(f"{'------':>6}  {'----------':>12}  {'-------':>8}")

for i in range(1, 11):
    action = Action(
        agent_id="notebook-agent",
        action_type="read",
        target="documents",
    )
    context = AgentContext(
        agent_id="notebook-agent",
        trust_profile=runtime.get_trust_profile("notebook-agent"),
    )
    verdict = runtime.evaluate(action, context)

    # Get the updated trust score after the evaluation
    trust = runtime.get_trust_profile("notebook-agent").overall_trust
    print(f"{i:>6}  {trust:>12.4f}  {verdict.verdict.name:>8}")

In [ ]:
# Boundary testing: attempt an action outside the configured scope
#
# We configured the agent's scope as {"read", "write", "query"}.
# A "delete" action is NOT in that scope, so the scope_compliance
# dimension will veto the action.

out_of_scope_action = Action(
    agent_id="notebook-agent",
    action_type="delete",
    target="documents",
)

context = AgentContext(
    agent_id="notebook-agent",
    trust_profile=runtime.get_trust_profile("notebook-agent"),
)

verdict = runtime.evaluate(out_of_scope_action, context)

print(f"Verdict:    {verdict.verdict.name}")
print(f"UCS Score:  {verdict.ucs:.4f}")
print(f"Tier:       {verdict.tier}")
print(f"Vetoed by:  {verdict.vetoed_by}")
print(f"Reasoning:  {verdict.reasoning}")
print()
print("The 'delete' action was denied because it falls outside the agent's")
print("configured scope of {read, write, query}. The scope_compliance dimension")
print("issues a veto -- a deterministic gate at Tier 1 that immediately denies")
print("the action without needing to consult higher tiers.")

In [ ]:
# Audit trail: inspect the structured log of every governance decision
#
# The audit trail is an append-only, hash-chained log that records
# every governance event with enough detail to reconstruct decisions
# after the fact.

records = runtime.audit_trail.query(agent_id="notebook-agent", limit=20)

print(f"Found {len(records)} audit records for 'notebook-agent'\n")
print(f"{'#':>3}  {'Action Type':<14}  {'Verdict':<10}  {'UCS':>6}  {'Tier':>4}")
print(f"{'---':>3}  {'-----------':<14}  {'-------':<10}  {'-----':>6}  {'----':>4}")

for i, record in enumerate(records, 1):
    print(
        f"{i:>3}  {record.action_type:<14}  {record.verdict:<10}  "
        f"{record.ucs:>6.3f}  {record.tier:>4}"
    )

## Next Steps

Now that you have seen the fundamentals of Nomotic governance, explore these notebooks for deeper topics:

- **`02-governed-tool-executor.ipynb`** -- Wrap tools with governance so every tool call is evaluated, monitored, and audited automatically.
- **`03-drift-detection.ipynb`** -- Detect behavioral drift: when an agent's actions deviate from its established fingerprint.
- **`04-adversarial-testing.ipynb`** -- Stress-test governance with adversarial scenarios, prompt injection attempts, and escalation attacks.

For full documentation, see the **`docs/`** directory in this repository.